# 의사결정나무

---

## Part1. 분류(Classification)

### 1. 분석 데이터 준비

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [3]:
data1 = pd.read_csv('data/breast-cancer-wisconsin.csv')
data1.head()

,code,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


In [7]:
# 특성데이터 분류
X = data1[data1.columns[1:10]]
y = data1[['Class']]

In [8]:
# 학습, 테스트 데이터 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify= y, random_state=42)

In [10]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

### 2.기본모델 적용 (의사결정나무)
- from sklearn.tree import DecisionTreeClassifier

In [11]:
from sklearn.tree import DecisionTreeClassifier 

In [13]:
model1 = DecisionTreeClassifier()
model1.fit(X_scaled_train, y_train)
pred_train = model1.predict(X_scaled_train)
model1.score(X_scaled_train, y_train)

1.0

In [14]:
# 오차행렬
from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(y_train, pred_train)
print('훈련데이터 오차행렬:\n', confusion_train)

훈련데이터 오차행렬:
 [[333   0]
 [  0 179]]


In [15]:
# 분류예측 레포트
from sklearn.metrics import classification_report
cfreport_train = classification_report(y_train, pred_train)
print('분류예측 레포트:\n', cfreport_train)

분류예측 레포트:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       333
           1       1.00      1.00      1.00       179

    accuracy                           1.00       512
   macro avg       1.00      1.00      1.00       512
weighted avg       1.00      1.00      1.00       512



In [18]:
# 테스트 데이터
pred_test=model1.predict(X_scaled_test)
model1.score(X_scaled_test, y_test)

0.9532163742690059

In [19]:
# 오차행렬
confusion_test=confusion_matrix(y_test, pred_test)
print("테스트데이터 오차행렬:\n", confusion_test)

테스트데이터 오차행렬:
 [[106   5]
 [  3  57]]


In [20]:
# 분류예측 레포트
cfreport_test=classification_report(y_test, pred_test)
print("분류예측 레포트:\n", cfreport_test)

분류예측 레포트:
               precision    recall  f1-score   support

           0       0.97      0.95      0.96       111
           1       0.92      0.95      0.93        60

    accuracy                           0.95       171
   macro avg       0.95      0.95      0.95       171
weighted avg       0.95      0.95      0.95       171



### 3.Grid Search

In [21]:
from sklearn.model_selection import GridSearchCV

In [22]:
param_grid = {'max_depth' : range(2,20,2), 'min_samples_leaf':range(1,50,2)}

In [23]:
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': range(2, 20, 2),
                       

In [24]:
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Score: {:.4f}".format(grid_search.best_score_))
print("TestSet Score: {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

Best Parameter: {'max_depth': 18, 'min_samples_leaf': 1}
Best Score: 0.9648
TestSet Score: 0.9532


### 4.Random Search

In [25]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [26]:
param_distribs = {'max_depth':randint(low=1, high=20),
                  'min_samples_leaf':randint(low=1, high=50)}

In [27]:
random_search = RandomizedSearchCV(DecisionTreeClassifier(),
                                   param_distribs,n_iter=20, cv=5)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=None,
          

In [28]:
print("Best Parameter: {}".format(random_search.best_params_))
print("Best Score: {:.4f}".format(random_search.best_score_))
print("TestSet Score: {:.4f}".format(random_search.score(X_scaled_test, y_test)))

Best Parameter: {'max_depth': 4, 'min_samples_leaf': 5}
Best Score: 0.9453
TestSet Score: 0.9474


---

## Part2.회귀(Regression)

### 1. 분석 데이터 준비

In [29]:
data2 = pd.read_csv('data/house_price.csv')
data2.head()

,housing_age,income,bedrooms,households,rooms,house_value
0,23,6.7770,0.141112,2.442244,8.103960,500000
1,49,6.0199,0.160984,2.726688,5.752412,500000
2,35,5.1155,0.249061,1.902676,3.888078,500000
3,32,4.7109,0.231383,1.913669,4.508393,500000
4,21,4.5625,0.255583,3.092664,4.667954,500000


In [32]:
# 특성데이터 분류
X = data2[data2.columns[1:5]]
y = data2[['house_value']]

In [33]:
# 훈련, 테스트 데이터 분류
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, random_state=42)

In [34]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(X_train)
X_scaled_train=scaler.transform(X_train)
X_scaled_test=scaler.transform(X_test)

### 2.기본모델적용 (의사결정나무)
- from sklearn.tree import DecisionTreeRegressor

In [35]:
from sklearn.tree import DecisionTreeRegressor

In [36]:
model = DecisionTreeRegressor()
model.fit(X_scaled_train, y_train)
pred_train=model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

1.0

In [37]:
# 테스트 데이터
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.21444149956334257

In [38]:
# RMSE (Root Mean Squared Error)
import numpy as np
from sklearn.metrics import mean_squared_error
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈련   데이터 RMSE:", np.sqrt(MSE_train))
print("테스트 데이터 RMSE:", np.sqrt(MSE_test))

훈련   데이터 RMSE: 0.0
테스트 데이터 RMSE: 84732.83554576062


### 3.Grid Search

In [39]:
from sklearn.model_selection import GridSearchCV

In [40]:
param_grid = {'max_depth':range(2,20,2), 'min_samples_leaf':range(1,50,2)}

In [41]:
grid_search = GridSearchCV(DecisionTreeRegressor(), param_grid, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=None,
                                             splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': range(2, 20, 2),
                         'min_samples_leaf': range(1, 50, 2)},
             pre_dispatch='2*n_jobs',

In [43]:
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Score: {:.4f}".format(grid_search.best_score_))
print("TestSet Score: {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

Best Parameter: {'max_depth': 8, 'min_samples_leaf': 49}
Best Score: 0.5592
TestSet Score: 0.5770


### 4.Random Search

In [44]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [45]:
param_distribs = {'max_depth':randint(low=1, high=20),
                  'min_samples_leaf':randint(low=1, high=50)}

In [46]:
random_search = RandomizedSearchCV(DecisionTreeRegressor(),
                                   param_distribs, n_iter=20, cv=5)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=DecisionTreeRegressor(ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features=None,
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   presort='deprecated',
                                                   random_state=None,
                                                   splitter='best'),
                   iid='d

In [49]:
print("Best Parameter: {}".format(random_search.best_params_))
print("Best Score: {:.4f}".format(random_search.best_score_))
print("TestSet Score: {:.4f}".format(random_search.score(X_scaled_test, y_test)))

Best Parameter: {'max_depth': 8, 'min_samples_leaf': 36}
Best Score: 0.5556
TestSet Score: 0.5752
